<a href="https://colab.research.google.com/github/RashmiLnu/aqa-test-tools/blob/Google_Collab_Rashmi/Version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain together openai faiss-cpu tiktoken


In [ ]:
import os
from langchain.llms import Together

# Paste your Together.ai API key here (inside quotes)
os.environ["TOGETHER_API_KEY"] = "Token_KEY"

# Load the LLM from Together.ai (you can switch models later)
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.3,
    max_tokens=512
)


In [ ]:
!pip install -q langchain-community

In [ ]:
!pip install -U langchain-together

In [ ]:
import os
from langchain_together import Together

# Paste your Together.ai API key here (inside quotes)
os.environ["TOGETHER_API_KEY"] = "Token"

# Load the LLM from Together.ai (you can switch models later)
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.3,
    max_tokens=512
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define prompt template
template = """
You are a debugging assistant.
Given the following test failure and commit diff, determine whether the commit likely caused the failure.

Test Failure:
{failure}

Commit Diff:
{diff}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["failure", "diff"],
    template=template,
)

test_failure = "NullPointerException at NullHandler.java:42"
commit_diff = """
diff --git a/NullHandler.java b/NullHandler.java
@@ -40,6 +40,7 @@
 public void handle() {
     Object obj = null;
     obj.toString();  // ← added line
 }
"""


chain = prompt | llm
result = chain.invoke({
    "failure": test_failure,
    "diff": commit_diff
})

print("🤖 GPT Response:\n", result)


In [ ]:
!pip install -q sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load open-source embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # or use bge-small-en

# Simulate commit diffs
commit_diffs = [
    ("sha1", "Added null check to NullHandler.java"),
    ("sha2", "Refactored config loading"),
    ("sha3", "Removed unused function from TestRunner"),
    ("sha4", "Introduced obj.toString() in NullHandler.java"),
    ("sha5", "Added logging in exception handler")
]

# Embed each diff
texts = [text for _, text in commit_diffs]
embeddings = embed_model.encode(texts, convert_to_numpy=True)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Store commit metadata separately
sha_lookup = [sha for sha, _ in commit_diffs]


In [ ]:
# Test failure text
test_failure_context = "NullPointerException at NullHandler.java:42 in handle()"

# Embed the failure
query_embedding = embed_model.encode([test_failure_context])

# Search for top 3 most similar diffs
D, I = index.search(query_embedding, 3)
top_matches = [(sha_lookup[i], commit_diffs[i][1]) for i in I[0]]

print("🔍 Top relevant commit diffs:\n")
for sha, diff in top_matches:
    print(f"Commit {sha}:\n{diff}\n")


In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are a debugging assistant.

A test failed with the following message:
{failure}

Here is a commit diff that was semantically similar:
{diff}

Do you think this commit caused the failure? Why or why not?
"""

prompt = PromptTemplate(
    input_variables=["failure", "diff"],
    template=template,
)

chain = prompt | llm


In [ ]:
print("🤖 GPT-style analysis:\n")

for sha, diff in top_matches:
    print(f"🧩 Commit {sha}:")
    response = chain.invoke({
        "failure": test_failure_context,
        "diff": diff
    })
    print(response)
    print("-" * 60)


In [ ]:
combined_diff_text = ""

for sha, diff in top_matches:
    combined_diff_text += f"--- Commit {sha} ---\n{diff}\n\n"


In [ ]:
from langchain.prompts import PromptTemplate

combined_template = """
You are a debugging assistant helping identify the root cause of a test failure.

Here is the test failure:
{failure}

Here are the top 3 semantically related commits:
{diffs}

Based on the diff content and the test failure, which commit is most likely responsible? Explain your reasoning.
"""

combined_prompt = PromptTemplate(
    input_variables=["failure", "diffs"],
    template=combined_template
)

# Create runnable chain
combined_chain = combined_prompt | llm


In [ ]:
result = combined_chain.invoke({
    "failure": test_failure_context,
    "diffs": combined_diff_text
})

print("🤖 Final Verdict:\n")
print(result)
